# Training scDiffusion-X

Clone the scDiffusion-X to your local machine:
```
git clone https://github.com/EperLuo/scDiffusion-X.git
cd scDiffusion-X
```
Set the conda environment. (Please refer to the Installation section)

**Data preparation**

The data for training scDiffusion-X should contain two different modalities and saved in moun ```h5mu``` format. For scRNA-seq data, the row count expression data should be saved in ```mdata['rna'].X```. For scATAC-seq data, the binary chromatin openness should be saved in ```mdata['atac'].X```. Meta-information such as cell type can be placed in ```mdata['rna'].obs['cell_type']```. See the example data in xxx for details.


**Train the Autoencoder**

After organizing the data, you can start to train the multi-modal Autoencoder. 
```
cd script/training_autoencoder
bash train_autoencoder_multimodal.sbatch
```
Adjust the data path to your local path. The dataset config file is in script/training_autoencoder/configs/dataset, see the comments in `openproblem.yaml` for details. The checkpoint will be saved in script/training_autoencoder/outputs/checkpoints and the log file will be saved in script/training_autoencoder/outputs/logs. The autoencoder config file is in script/training_autoencoder/configs/encoder, see the comments in `encoder_multimodal.yaml` for details. 

There are three different sizes Autoencoders: `encoder_multimodal_small`, `encoder_multimodal`, and `encoder_multimodal_large`. We recommand to use `encoder_multimodal` (corresponding to `encoder_multimodal.yaml`) for most of dataset. If the genes and peaks are more than 50,000 and 200,000, we recommand a larger autoencoder in `encoder_multimodal_large.yaml`. If the genes and peaks are less than 5,000 and 15,000, we recommand a smaller autoencoder in `encoder_multimodal_small.yaml`. The `norm_type` in the encoder config yaml control the normalization type. For data generation task, we recommend `batch_norm`, and for translation task, we recommend `layer_norm` since it has better generalization for OOD data.


**Train the diffusion backbone**

```
cd script/training_diffusion
sh ssh_scripts/multimodal_train.sh
```
Again, adjust the data path and output path to your own, and also change the ae_path&encoder_config in the sh file to the autoencoder you tarined in step 1. The `rna_dim` and `atac_dim` refer to the dimensions of latent representation, you should change them to match the autoencoder you used (refer to the encoder's config file). When training with condition (like the cell type condition), set the `num_class` to the number of unique labels. The training is unconditional when the `num_class` is not set. 

Also, change the `devices` and `NUM_GPUS` parameter according to your own situation. The total batch size is num_gpu*batch_size. 
